Name: Tchangmena A Nken Allassan

Creation Date: 31/10/2021

Update Date: 31/10/2021

We use a GPU to train the model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import necessary libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import pandas as pd
import keras
import cv2
from tqdm import  tqdm
import os
from torchvision import transforms

In [3]:
from PIL import Image

In [4]:
#unzipping the data files
!unzip "drive/MyDrive/Train_coral.zip" -d "Train_coral/"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: Train_coral/Other_Invertebrates/augmented_image_119.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_2754.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_7289.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_3613.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_6044.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_3636.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_5405.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_1793.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_1118.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_1076.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_9637.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_image_4777.jpg  
  inflating: Train_coral/Other_Invertebrates/augmented_imag

In [5]:
!unzip "drive/MyDrive/Test_coral.zip" -d "Test_coral/"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: Test_coral/ImageID_K07NKATM.jpg  
  inflating: Test_coral/ImageID_LK6QDQOD.jpg  
  inflating: Test_coral/ImageID_8A3I2R6P.jpg  
  inflating: Test_coral/ImageID_V4EMJG4H.jpg  
  inflating: Test_coral/ImageID_TX8T4YB8.jpg  
  inflating: Test_coral/ImageID_HY3S2IEZ.jpg  
  inflating: Test_coral/ImageID_Y7CK72OJ.jpg  
  inflating: Test_coral/ImageID_A44W393H.jpg  
  inflating: Test_coral/ImageID_SBUQS0LR.jpg  
  inflating: Test_coral/ImageID_9TDLCV4O.jpg  
  inflating: Test_coral/ImageID_46REAQ5S.jpg  
  inflating: Test_coral/ImageID_GCYO7XZ4.jpg  
  inflating: Test_coral/ImageID_HEOB3H9U.jpg  
  inflating: Test_coral/ImageID_Z8FUKSUT.jpg  
  inflating: Test_coral/ImageID_D1QT09OB.jpg  
  inflating: Test_coral/ImageID_LW60W2IW.jpg  
  inflating: Test_coral/ImageID_T3C8J0ES.jpg  
  inflating: Test_coral/ImageID_UYRXTKQM.jpg  
  inflating: Test_coral/ImageID_VIH4PPMJ.jpg  
  inflating: Test_coral/Image

## Data Preparation

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
BATCH_SIZE=256
IMAGE_SIZE=(224,224,3)
train_path="Train_coral/"

In [8]:
train_generator=ImageDataGenerator(rescale=1./255,
                                  validation_split=0.2)

train_data=train_generator.flow_from_directory(train_path,
                                               target_size=(224,224),
                                               color_mode="rgb",
                                               class_mode="categorical",
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               subset='training')

val_data=train_generator.flow_from_directory(train_path,
                                               target_size=(224,224),
                                               color_mode="rgb",
                                               class_mode="categorical",
                                               batch_size=BATCH_SIZE,
                                               shuffle=False,
                                               subset='validation')

Found 43749 images belonging to 5 classes.
Found 10935 images belonging to 5 classes.


In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
base_model=VGG16(weights="imagenet",include_top=False,input_tensor=Input(shape=IMAGE_SIZE))

head_model=base_model.output
head_model=Flatten(name="flatten")(head_model)
head_model=Dense(4096,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(4096,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(5,activation='softmax')(head_model)

for layers in base_model.layers:
  layers.trainable=False

model=Model(inputs=base_model.input,outputs=head_model)
opt=Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])

58900480/58889256 [==============================] - 1s 0us/step


In [11]:
early_stop=EarlyStopping(patience=2,monitor='val_loss')

In [12]:
result=model.fit_generator(train_data,epochs=3,validation_data=val_data,
                           callbacks=[early_stop])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
171/171 [==============================] - 565s 3s/step - loss: 2.1797 - accuracy: 0.3841 - val_loss: 1.4424 - val_accuracy: 0.3692
Epoch 2/3
171/171 [==============================] - 400s 2s/step - loss: 1.2802 - accuracy: 0.4519 - val_loss: 1.4305 - val_accuracy: 0.3893
Epoch 3/3
171/171 [==============================] - 400s 2s/step - loss: 1.2316 - accuracy: 0.4713 - val_loss: 1.4621 - val_accuracy: 0.4073


In [10]:
model.save('koubrah_coral.h5')

NameError: ignored

In [11]:
model = tf.keras.models.load_model("drive/MyDrive/koubrah_coral.h5")

In [12]:
mapping_class = val_data.class_indices

In [13]:
mapping_class

{'Algae': 0,
 'Hard_Coral': 1,
 'Other': 2,
 'Other_Invertebrates': 3,
 'Soft_Coral': 4}

In [14]:
mapping_class = train_data.class_indices


In [10]:
mapping_class

{'Algae': 0,
 'Hard_Coral': 1,
 'Other': 2,
 'Other_Invertebrates': 3,
 'Soft_Coral': 4}

In [15]:
mapping_class = dict([(value, key) for key, value in mapping_class.items()]) 

In [ ]:
pred_label = mapping_class[np.argmax(pred_prob)]

In [19]:
sub=pd.read_csv('SampleSubmission.csv')

In [20]:
test_path='Test_coral/'

In [21]:
image_transform=transforms.Compose([
        transforms.Resize(size=224)
    ])

In [22]:
test_img=[]
for i in range(0,sub.shape[0]) :
    img_path=test_path+str(sub.iloc[i,:].Image_ID)+".jpg"
    image = Image.open(img_path).convert('RGB')
    image=image_transform(image)
    image = np.array(image)
    test_img.append((image))

In [16]:
test_img

[array([[[ 70,  66,  57],
         [ 71,  67,  58],
         [ 72,  68,  61],
         ...,
         [ 97,  96, 102],
         [104, 103, 109],
         [107, 106, 112]],
 
        [[ 71,  67,  58],
         [ 72,  68,  59],
         [ 74,  70,  62],
         ...,
         [ 96,  96, 103],
         [104, 104, 110],
         [107, 107, 114]],
 
        [[ 74,  70,  59],
         [ 75,  71,  61],
         [ 78,  74,  65],
         ...,
         [ 95,  96, 104],
         [105, 106, 114],
         [108, 110, 118]],
 
        ...,
 
        [[244, 248, 247],
         [244, 248, 247],
         [244, 248, 247],
         ...,
         [180, 181, 148],
         [170, 172, 135],
         [166, 168, 130]],
 
        [[245, 249, 248],
         [245, 249, 248],
         [245, 249, 248],
         ...,
         [197, 198, 166],
         [187, 189, 154],
         [184, 186, 149]],
 
        [[245, 249, 248],
         [245, 249, 248],
         [245, 249, 248],
         ...,
         [203, 204, 173],
  

In [ ]:
test_img=np.array(test_img)*(1./255)

In [1]:
sub.label=predictions

NameError: ignored

In [19]:
submit.tail()

,Image_ID,label
11958,ImageID_ZZP1Y9Z1,NaN
11959,ImageID_ZZPHNP5L,NaN
11960,ImageID_ZZUIMD0P,NaN
11961,ImageID_ZZUXL8E9,NaN
11962,ImageID_ZZWZ8V51,NaN
